In [6]:
import os
import json
from dotenv import load_dotenv
import gradio as gr
import json
from openai import OpenAI
import re

In [7]:
load_dotenv(override=True)
api_key = os.getenv('OPENAI_API_KEY')
    
client = OpenAI()

In [8]:
system_prompt = """
    You are an expert technical writer and knowledge engineer.
    Your task is to generate well-structured Markdown (.md) documentation files that can be used as a knowledge base for a RAG.

    Follow these rules carefully:
    1. Write the content in clear, concise Markdown format.
    2. Use appropriate Markdown headers (#, ##, ###) to structure the document.
    3. Include lists, tables, or code blocks only when necessary.
    4. Keep each document self-contained and focused on a single topic.
    5. Do not include any text outside the Markdown content (no explanations, no code fences).
    6. The style should be factual, structured, and helpful for machine retrieval.
    7. Use consistent tone and terminology across sections.
    """

In [9]:
def create_kb_prompt(topic, kb_type="tutorial"):
    return f"""
    Generate a comprehensive Markdown document for the following technical topic.
    Topic: {topic}
    Document Type: {kb_type}
    The document should include structured sections, concise explanations, and clear formatting suitable for a technical knowledge base.
    """

In [10]:
def generate_markdown_doc(topic, kb_type="tutorial"):
    
    user_prompt = create_kb_prompt(topic, kb_type)
    messages = [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": user_prompt},
    ]
    
    response = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=messages,
        temperature=0.7
    )
    markdown_output = response.choices[0].message.content.strip()
    markdown_output = re.sub(r'^```[a-z]*\\s*', '', markdown_output, flags=re.MULTILINE)
    markdown_output = re.sub(r'\\s*```$', '', markdown_output, flags=re.MULTILINE)
    return markdown_output

In [ ]:
def create_kb_gradio_interface():
    with gr.Blocks(theme=gr.themes.Soft()) as app:
        gr.Markdown("## Technical Knowledge Base Generator")

        with gr.Row():
            with gr.Column():
                topic_input = gr.Textbox(
                    label="Technical Topic",
                    placeholder="e.g., Building a RAG pipeline with LangChain...",
                    lines=2
                )
                kb_type_input = gr.Radio(
                    label="Document Type",
                    choices=["Overview", "FAQ", "Use Case"],
                    value="FAQ"
                )
                generate_button = gr.Button("Generate Markdown Document", variant="primary")

            with gr.Column():
                output_md = gr.Textbox(
                    label="Generated Markdown Content",
                    lines=25,
                    interactive=False,
                    placeholder="Generated Markdown will appear here..."
                )

        generate_button.click(
            fn=generate_markdown_doc,
            inputs=[topic_input, kb_type_input],
            outputs=[output_md],
            api_name="generate_kb_doc"
        )

    return app

In [ ]:
app = create_kb_gradio_interface()
app.launch(debug=True, share=True)